In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
resto = pd.read_csv('Top250.csv')

# Data Preparation
Quick look at the data, and its data types and the null values present 

Display first 5 values 

In [3]:
resto.head()

Rank   Restaurant                                            Content  \
0     1   McDonald's                                                NaN   
1     2    Starbucks                                                NaN   
2     3  Chick-fil-A  While Popeyes got a lot of the chicken buzz in...   
3     4    Taco Bell                                                NaN   
4     5  Burger King                                                NaN   

   Sales YOY_Sales  Units YOY_Units Headquarters             Segment_Category  
0  40412      4.9%  13846     -0.5%          NaN       Quick Service & Burger  
1  21380      8.6%  15049      3.0%          NaN  Quick Service & Coffee Cafe  
2  11320     13.0%   2470      5.0%          NaN      Quick Service & Chicken  
3  11293      9.0%   6766      2.7%          NaN      Quick Service & Mexican  
4  10204      2.7%   7346      0.2%          NaN       Quick Service & Burger

Display data for each column

In [4]:
resto.dtypes

Rank                 int64
Restaurant          object
Content             object
Sales                int64
YOY_Sales           object
Units                int64
YOY_Units           object
Headquarters        object
Segment_Category    object
dtype: object

Check for null values

In [5]:
resto.isnull()

Rank  Restaurant  Content  Sales  YOY_Sales  Units  YOY_Units  \
0    False       False     True  False      False  False      False   
1    False       False     True  False      False  False      False   
2    False       False    False  False      False  False      False   
3    False       False     True  False      False  False      False   
4    False       False     True  False      False  False      False   
..     ...         ...      ...    ...        ...    ...        ...   
245  False       False     True  False      False  False      False   
246  False       False     True  False      False  False      False   
247  False       False     True  False      False  False      False   
248  False       False     True  False      False  False      False   
249  False       False     True  False      False  False      False   

     Headquarters  Segment_Category  
0            True             False  
1            True             False  
2            True             False  
3            True             False  
4            True             False  
..            ...               ...  
245          True             False  
246          True             False  
247          True             False  
248          True             False  
249          True             False  

[250 rows x 9 columns]

# Data Cleaning
Remove columns with null values and convert data to its correct type

Remove columns content and headquarters since it contains null values

In [6]:
resto_cleaned = resto.drop(['Content', 'Headquarters'], axis=1)

Remove percentage symbol 

In [7]:
resto_cleaned['YOY_Sales'] = list(map(lambda x : x[:-1], resto_cleaned['YOY_Sales']))
resto_cleaned['YOY_Units'] = list(map(lambda x : x[:-1], resto_cleaned['YOY_Units']))

Convert YOY_Sales and YOY_Units to float

In [8]:
resto_cleaned['YOY_Sales'] = list(map(float, resto_cleaned['YOY_Sales']))
resto_cleaned['YOY_Units'] = list(map(float, resto_cleaned['YOY_Units']))

Rename columns to clarify that the numbers are in percentage

In [9]:
resto_cleaned = resto_cleaned.rename(columns={'YOY_Sales': 'YOY_Sales in (%)', 'YOY_Units': 'YOY_Units in (%)'})

In [10]:
resto_cleaned.head()

Rank   Restaurant  Sales  YOY_Sales in (%)  Units  YOY_Units in (%)  \
0     1   McDonald's  40412               4.9  13846              -0.5   
1     2    Starbucks  21380               8.6  15049               3.0   
2     3  Chick-fil-A  11320              13.0   2470               5.0   
3     4    Taco Bell  11293               9.0   6766               2.7   
4     5  Burger King  10204               2.7   7346               0.2   

              Segment_Category  
0       Quick Service & Burger  
1  Quick Service & Coffee Cafe  
2      Quick Service & Chicken  
3      Quick Service & Mexican  
4       Quick Service & Burger

# Data Analysis

## Statistics

Basic statistical analysis for quantitative data

### Observation


In [11]:
resto_cleaned[['Sales', 'Units']].describe()

Sales         Units
count    250.00000    250.000000
mean    1242.74000    850.076000
std     3365.22882   2296.151659
min      126.00000     13.000000
25%      181.00000     85.000000
50%      330.00000    207.000000
75%      724.75000    555.250000
max    40412.00000  23801.000000

In [12]:
resto_cleaned[['YOY_Sales in (%)', 'YOY_Units in (%)']].describe()

YOY_Sales in (%)  YOY_Units in (%)
count        250.000000        250.000000
mean           2.938400          1.219200
std            8.953784          7.936431
min          -21.200000        -32.800000
25%           -2.375000         -2.025000
50%            2.200000          0.000000
75%            6.575000          3.475000
max           39.500000         38.500000

Showing top 10 categories with highest frequency

In [13]:
# resto_cleaned.groupby('Segment_Category').count()
resto_cleaned['Segment_Category'].value_counts().head(10)

Varied Menu                    22
Mexican                        14
Quick Service & Burger         13
Burger                         10
Italian/Pizza                  10
Family Style                   10
Casual Dining & Varied Menu     9
Steak                           9
Pizza                           9
Seafood                         8
Name: Segment_Category, dtype: int64

Most categories that have the most sales are under the quick service category  

In [14]:
sales_by_segment_categ = resto_cleaned.groupby('Segment_Category').agg({ 'Sales': 'sum'}).sort_values(by='Sales', ascending=False)
sales_by_segment_categ.head(10) 

Sales
Segment_Category                   
Quick Service & Burger        79384
Quick Service & Coffee Cafe   31889
Quick Service & Chicken       22619
Quick Service & Pizza         21316
Casual Dining & Varied Menu   15686
Quick Service & Sandwich      14965
Quick Service & Mexican       12143
Family Dining & Family Style  11677
Casual Dining & Steak          8014
Fast Casual & Mexican          7159

Split Segment_Category into category and add column Category

In [15]:
split_categ = resto_cleaned['Segment_Category'].str.split("&", n = -1, expand = True)
resto_cleaned['Category'] = split_categ[0]

In [16]:
sales_by_categ = resto_cleaned.groupby('Category').agg({ 'Sales': 'sum'}).sort_values(by='Sales', ascending=False)
sales_by_categ.head(10) 

Sales
Category              
Quick Service   192513
Casual Dining    38689
Fast Casual      32699
Family Dining    11677
Varied Menu       4610
Mexican           3396
Steak             2989
Pizza             2585
Family Style      2292
Burger            2187